In [16]:
import torch
import torchaudio
from torch import nn
from torch.nn import functional as F
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [11]:
torch.cuda.is_available()
torch.cuda.device_count()
torch.cuda.current_device()
torch.cuda.device(0)
torch.cuda.get_device_name(0)

'NVIDIA GeForce RTX 3060'

In [ ]:
NOISY_SAMPLES = os.path.join()

In [12]:
class Denoiser(torch.nn.Module):
  # D: no of encoder layers
  # H: no. of output channels in first layer
  # K: kernel size
  # S: stride
  def __init__(
  self, 
  n_layers, 
  output_channels, 
  chin=1,
  chout=1,
  hidden=48,
  depth=5,
  kernel_size=8,
  stride=4,
  causal=True,
  resample=4,
  growth=2,
  max_hidden=10_000,
  normalize=True,
  glu=True,
  rescale=0.1,
  floor=1e-3,
  sample_rate=22_050
  ):
    super(Denoiser, self).__init__()
    self.D = n_layers
    self.H = output_channels
    self.K = kernel_size
    self.S = kernel_size // 2

    self.encoder = nn.ModuleList([])
    self.decoder = nn.ModuleList([])
    activation = nn.GLU(1) if glu else nn.ReLU()
    ch_scale = 2 if glu else 1

    for index in range(depth):

      encode = []
      encode += [
          nn.Conv1d(chin, hidden, kernel_size, stride),
          nn.ReLU(),
          nn.Conv1d(hidden, hidden * ch_scale, 1),
          activation,
      ]
      self.encoder.append(nn.Sequential(*encode))

      decode = []
      decode += [
          nn.Conv1d(hidden, ch_scale * hidden, 1),
          activation,
          nn.ConvTranspose1d(hidden, chout, kernel_size, stride),
      ]
      if index > 0:
          decode.append(nn.ReLU())
      self.decoder.insert(0, nn.Sequential(*decode))
      chout = hidden
      chin = hidden
      hidden = min(int(growth * hidden), max_hidden)

  def forward(self, input):
    length = input.shape[-1]
    skip_outputs = []
    for encoder in self.encoder:
      x = encoder(x)
      skip_outputs.append(x)
      



0

'NVIDIA GeForce RTX 3060'